# **The Application of Deep Reinforcement Learning in Snake Game**

Team member: Shujie Cao, Lihui Yi, Jiaqi Zang

---

## **Introduction**
In recent years, Deep Reinforcement Learning (DRL) has gained significant attention, demonstrating superhuman performances in complex environments such as video games, robot control, etc.

In this project, we apply DRL concepts to a classic game, Snake Game. The game of Snake involves controlling a snake in a grid, trying to eat food while avoiding collisions with the snake's own body and the grid borders. The player's score is determined by the amount of food the snake has eaten. The primary goal of our project is to train an AI agent to play this game and maximize the score.

We begin by implementing the game of Snake, ensuring we have a well-defined environment with clear state representations, actions, and rewards - critical components of any RL system. We then use Deep Q-Network (DQN) and Double DQN (DDQN) to train our agent, which are both powerful and well-known algorithms that show great performance in many tasks.

DQN and DDQN extend traditional Q-Learning by using deep neural networks as function approximators, allowing the agent to handle environments with high-dimensional state spaces. Additionally, in DDQN, in order to reduce the overestimation bias of Q-values, another mechanism is included. It improves the stability and performance of the learning process.

Through this project, we not only learned to apply the DRL methods taught in class, but also had the opportunity to face some practical challenges. For example, some way of defining state and action spaces will significantly slow down the learning process, making the task unable to finish in a personal laptop. For details like this, as well as the questions asked in course syllabus, we will have a discussion in Part III.

Note that it would be easier to get the idea of our model and big pictures via our video. In this notebook, we mainly talk about the details. Also, since we use the pygame library that Jupyter doesn't support, this Jupyter notebook might not be run successfully. In order to run the code, please copy and paste in a python file. The requirements are up to date packages. Our video link is: https://youtu.be/aTUhJUjlg3I

In [ ]:
import math
import pygame
import random
import numpy as np
from datetime import datetime
from collections import deque
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

## **Part I: Snake Game**


---
In the following few blocks, we implement this classic snake game by "pygame" library. The snake game we wrote follows the same rule as the one we usually see online. We use class attribute in python for implementation. This class encapsulates all the functionality needed to play the game of Snake.

> First, we initialize the Snake Game instance. By default, the game UI canvas is a 16 × 12 grid. After initialization, we call function "reset" to prepare the game.



In [ ]:
# Define a snake game class
class SnakeGame:
    def __init__(self, width=640, height=480, grid_size=40):
        self.width = width
        self.height = height
        self.grid_size = grid_size
        self.reset()



> This method is responsible for resetting or starting a new game. It initializes the Pygame display, sets the game clock, and initializes the snake's starting position and food's position. Without loss of generality, we assume the snake always start around the middle of the grid, while the food is positioned randomly (can even be the same as snake). The output of the function is the initial state at the Genesis time. 

In [ ]:
    # Function to reset/restart the game
    def reset(self):
        # Initialize pygame
        pygame.init()
        pygame.display.set_caption('Snake Game')
        self.clock = pygame.time.Clock()
        self.screen = pygame.display.set_mode((self.width, self.height))
        self.font = pygame.font.SysFont('Arial', 30)
        self.score = 0

        # Initialize snake, the initial position is fixed for all games
        self.snake = deque()
        self.snake.append((self.width / 2, self.height / 2))

        # Initialize food
        self.food = self.get_random_pos()

        # Return the initial state
        return self.get_state(0)



> This method is used to generate a completely random position for the food on the game grid, without caring about whether the food sits on the snake head or snake body.

In [ ]:
    # Function to assign a random position to the food
    def get_random_pos(self):
        x = random.randint(0, self.width - self.grid_size)
        y = random.randint(0, self.height - self.grid_size)
        return (x // self.grid_size * self.grid_size, y // self.grid_size * self.grid_size)



> This function advances the game by one step, given an action input. The action is an integer from {0, 1, 2, 3}, representing the direction the snake moves. First, it updates the position of the snake head. Then, it checks whether the snake hits the borders or itself. If so, the snake dies in this round and a reward of -100 is given.

 > Afterwards, it checks whether the snake eats the food. If so, a reward of +10 is given. If not, it further checks if the snake head is getting closer to the food. If yes, then a small positive reward of +1 is given. Otherwise, a reward of -1 is given.

 > The reason we assigns the rewards of -100 and +10 is straightforward, because we want the snake to learn to eat the food and avoid a death. As for the ±1 reward, they are also important and necessary because they encourage the snake to explore the paths and look for the food. We will talk about what happens if these two rewards are removed in Part III.

 > Different from the reward, if and only if the snake eats the food, we add one score to this game.

 > At the end of this function, we update the snake body. The way we do that is not moving the entire snake by one step. Instead, we simply add another cell as the new snake head and remove the snake tail. In this way, the code can be simplified a little bit. 

 > Finally, the function returns a tuple containing a boolean indicating whether the game is still running, the current score, and a reward.

In [ ]:
    # Function to take a step in the game
    def step(self, action):
        # Action space: 0 - UP, 1 - RIGHT, 2 - DOWN, 3 - LEFT

        # Initialize the reward
        reward = 0

        # Get the current position of the snake head
        x, y = self.snake[0]

        # Update the position of the snake head
        if action == 0:
            y -= self.grid_size
        elif action == 1:
            x += self.grid_size
        elif action == 2:
            y += self.grid_size
        elif action == 3:
            x -= self.grid_size

        # Check if the snake is dead, if dead, reward is -200
        if x < 0 or x >= self.width or y < 0 or y >= self.height or (x, y) in self.snake:
            reward = -100
            return False, self.score, reward

        # Update the position of the snake
        self.snake.appendleft((x, y))

        # Check if the snake eats the food
        if (x, y) == self.food:
            reward = 10
            self.score += 1
            self.food = self.get_random_pos()
            # Check if the new food is on the snake, if so, assign a new position to the food
            while self.food in self.snake:
                self.food = self.get_random_pos()

        # else if the snake gets closer to the food, if so, reward is 1 and remove the snake tail
        elif (x - self.food[0]) ** 2 + (y - self.food[1]) ** 2 < (self.snake[1][0] - self.food[0]) ** 2 + (self.snake[1][1] - self.food[1]) ** 2:
            reward = 1
            self.snake.pop()

        # else if the snake gets farther to the food, if so, reward is -1 and remove the snake tail
        else:
            reward = -1
            self.snake.pop()

        return True, self.score, reward

## **Part II: Deep reinforcement training**


---



> In the following blocks, we will give the codes for the training using two models: Deep Q-Network and Double Deep Q-Network. First, we define a DQN policy network with three Hidden Layers.


In [ ]:
# Define a DQN agent
def OurModel(input_shape, action_space):
    X_input = Input(input_shape)

    # Input Layer of state size 12 and Hidden Layer with 128 nodes
    X = Dense(128, input_shape=input_shape, activation="relu", )(X_input)

    # Hidden layer with 128 nodes
    X = Dense(128, activation="relu")(X)

    # Hidden layer with 128 nodes
    X = Dense(128, activation="relu")(X)

    # Output Layer with # of actions: 4 nodes (up, right, down, left)
    X = Dense(action_space, activation="softmax")(X)

    model = Model(inputs=X_input, outputs=X, name='SnakeGame_DQN_model')
    model.compile(loss="mse", optimizer=Adam(learning_rate=0.00025), metrics=["accuracy"])

    # model.summary()
    return model

> We start our DQN agent by setting up some parameters.

In [ ]:
# Define a DQN agent
class DQNAgent:
    def __init__(self):
        self.env = SnakeGame()
        self.state_size = 13
        self.action_size = 4
        self.EPISODES = 50
        self.memory = deque(maxlen=2000)

        self.gamma = 0.95  # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.batch_size = 500
        self.train_start = 500

        # create main model
        self.model = OurModel(input_shape=(self.state_size,), action_space=self.action_size)

> The remember method is responsible for storing the agent's experiences in a memory buffer and gradually reducing the exploration rate as the agent gains more experience and begins training.

In [ ]:
    # Function to remember the state, the action, the reward, and the next state
    def remember(self, state, action, reward, next_state, running):
        self.memory.append((state, action, reward, next_state, running))
        if len(self.memory) > self.train_start:
            if self.epsilon > self.epsilon_min:
                self.epsilon *= self.epsilon_decay

> The act method implements an exploration-exploitation strategy for the snake agent. It randomly selects actions during the exploration phase, avoiding actions that make the snake go back. During the exploitation phase, it selects the action with the highest predicted Q-value, again avoiding actions that make the snake go back.

In [ ]:
    # Function to pick an action given a state, note that the snake is not allowed to go back
    def act(self, state):
        while True:
            # exploration
            if np.random.rand() <= self.epsilon:
                action_back = random.randrange(self.action_size)
                if action_back == 0 and state[0][2] == 1:
                    continue
                elif action_back == 1 and state[0][3] == 1:
                    continue
                elif action_back == 2 and state[0][0] == 1:
                    continue
                elif action_back == 3 and state[0][1] == 1:
                    continue
                else:
                    return action_back
            # exploitation
            else:
                action_back = np.argmax(self.model.predict(state, verbose=0)[0])
                if action_back == 0 and state[0][2] == 1:
                    action_back = random.randrange(self.action_size)
                    while action_back == 0:
                        action_back = random.randrange(self.action_size)
                elif action_back == 1 and state[0][3] == 1:
                    action_back = random.randrange(self.action_size)
                    while action_back == 1:
                        action_back = random.randrange(self.action_size)
                elif action_back == 2 and state[0][0] == 1:
                    action_back = random.randrange(self.action_size)
                    while action_back == 2:
                        action_back = random.randrange(self.action_size)
                elif action_back == 3 and state[0][1] == 1:
                    action_back = random.randrange(self.action_size)
                    while action_back == 3:
                        action_back = random.randrange(self.action_size)

                return action_back

> The replay method implements the experience replay mechanism in the DQN algorithm. It samples a mini-batch of experiences from the agent's memory, computes the target Q-values for each experience, and trains the DQN model using the states and target Q-values from the minibatch. This replay process helps stabilize and improve the learning process of the DQN agent.

In [ ]:
    # Function to replay the memory and train the network
    def replay(self):
        if len(self.memory) < self.train_start:
            return

        # Randomly sample minibatch from the memory
        minibatch = random.sample(self.memory, min(len(self.memory), self.batch_size))
        state = np.zeros((self.batch_size, self.state_size))
        next_state = np.zeros((self.batch_size, self.state_size))
        action, reward, running = [], [], []

        # assign data into state, next_state, action, reward and running from minibatch
        for i in range(self.batch_size):
            state[i] = minibatch[i][0]
            action.append(minibatch[i][1])
            reward.append(minibatch[i][2])
            next_state[i] = minibatch[i][3]
            running.append(minibatch[i][4])

        # Compute the value function of current and next state
        target = self.model.predict(state, verbose=0)
        target_next = self.model.predict(next_state, verbose=0)

        for i in range(self.batch_size):
            if not running[i]:
                target[i][action[i]] = reward[i]
            else:
                target[i][action[i]] = reward[i] + self.gamma * (np.amax(target_next[i]))

        # Train the Neural Network with batches where target is the value function
        self.model.fit(state, target, batch_size=self.batch_size, verbose=0)

 > The load() method loads a pre-trained DQN model from a file, and the save() method saves the current DQN model to a file. These methods enable the agent to persist the model's learned knowledge and reuse it for testing or further training at a later time without having to train the model from scratch.

In [ ]:
    # Function to load the model for testing purpose
    def load(self, name):
        self.model = load_model(name)

    # Function to save the model for testing purpose
    def save(self, name):
        self.model.save(name)

> The training method executes the training process for the DQN agent on a snake game environment. It iterates over episodes, interacts with the environment, stores experiences in memory, performs model training through replay, and saves the trained model at the end of the training process.






In [ ]:
    # Function to train the model
    def training(self):
        total_episodes = []
        total_rewards = []
        total_scores = []
        for e in range(self.EPISODES):
            state = self.env.reset()
            state = np.reshape(state, [1, self.state_size])
            running = True
            i = 0
            while running:
                action = self.act(state)
                running, score, reward, next_state = self.env.run(action)

                next_state = np.reshape(next_state, [1, self.state_size])

                self.remember(state, action, reward, next_state, running)
                state = next_state

                i += reward
                if not running:
                    dateTimeObj = datetime.now()
                    timestampStr = dateTimeObj.strftime("%H:%M:%S")
                    print("episode: {}/{}, score: {}, total reward: {}, e: {:.2}, time: {}".format(e + 1, self.EPISODES, score, i, self.epsilon,
                                                                                 timestampStr))
                    total_episodes.append(e + 1)
                    total_rewards.append(i)
                    total_scores.append(score)

                self.replay()

        # Save the trained model
        print("Saving trained model as SnakeGame-dqn-training.h5")
        self.save("SnakeGame-dqn-training.h5")

        return total_episodes, total_rewards, total_scores

> The test method evaluates the performance of the trained DQN agent on a snake game environment using the saved model. It loads the model, interacts with the environment using the model's predictions to choose actions, and prints the rewards achieved in each episode.

In [ ]:
    # test function if you want to test the learned model
    def test(self):
        self.load("SnakeGame-dqn-training.h5")
        for e in range(self.EPISODES):
            state = self.env.reset()
            state = np.reshape(state, [1, self.state_size])
            running = True
            i = 0
            while running:
                action = np.argmax(self.model.predict(state, verbose=0))
                running, score, reward, next_state = self.env.run(action)
                state = np.reshape(next_state, [1, self.state_size])
                i += reward

                if not running:
                    print("episode: {}/{}, reward: {}".format(e + 1, self.EPISODES, i))
                    break

> Since DDQN and DQN are similar, some of the codes are the same. Thus, we briefly talk about their similarity and difference rather than explaining the codes function by function like DQN. 

> First, both DQN and DDQN make use of deep neural networks to estimate the Q-values of state-action pairs, which represent the "quality" of taking a particular action in a given state. The difference between them is that how they handle the common issue of overestimation in Q-learning algorithms.

> In DQN, the same network is used for both the selection of the action and the generation of the Q-value target during the training. This means that DQN tends to overestimate the Q-values due to the max operator in the Bellman equation, which can lead to instability in training and suboptimal performance.

> To address this, DDQN uses a different approach. Instead of using a single network, DDQN utilizes two networks: the online network for action selection and the target network for generating the Q-value target. This small difference can substantially reduce the overestimation bias and leads to improved and more stable performance in many problems. Our numerical experiment results in video also revealed such feature.

> Due to page limit and meanwhile keep the completeness of our project, we leave the DDQN codes at the end of this report as an appendix.

In [ ]:
if __name__ == "__main__":
    agent_DQN = DQNAgent()
    episodes_DQN, rewards_DQN, scores_DQN = agent_DQN.training()

    # agent_DDQN = DDQNAgent()
    # episodes_DDQN, rewards_DDQN, scores_DDQN = agent_DDQN.training()

    agent_DQN.test()

## **Part III: Discussion**

---
In this part, we discuss the performances, model selections and parameters of DQN/DDQN algorithms in the Snake Game by answering the questions asked in the course syllabus. 


1.   *How do the algorithms perform in terms of training time?*
> In the current model settings, both algorithms perform very well in terms of training time. Specifically, it takes only 5min (20 episodes) to train the AI to have a pretty "reasonable" action, meaning that the snake begins to not touch the borders and try to go straight to the food. After training for 25min (50 episodes), the snake already knows how to avoid eating itself and meanwhile pick the shortest path to eat the food. After 1 day (1000 episodes) of training, our AI snake can even eat 1/3 of the maximum possible number of food, which our team members cannot achieve by playing manually. 

2.   *How close are the results to expected results?*
> Our expectation of the results is that the AI snake should play better than us. Experiments show that our AI snake indeed has already achieved this expectation. However, in term of the theoretical bound, this snake game is in fact mathematically solvable. It is an instance of the Hamiltonian cycle problem. Therefore, the optimal result is that the snake can grow until it full fills the entire grid. However, it is hard to achieve that theoretical optimal result. Because it needs the player to not be smart but be completely rational. In other words, the snake can always do the following: go right to the right border, go down by one step, go left to the left border, go down by one step... Though this snake can eat the food eventually, it has a very low efficiency and that solution is not "smart" or interesting.

3.   *How did you adjust the parameters of your algorithm to solve the problem?*
4.   *What parameters played an important role in solving the problem?*
5.   *What were challenges when working with the algorithms?*
> We answer these three questions together. The parameters and model in our problem are carefully designed. We also tried some other thoughts on the parameter and model. Among all those trials, the current setting is the best one. 

 > As a first attempt, we tried to use an ideal definition of state space, which is to let the state be the coordinates of snake head, snake body and food. However, that didn't work well because there are millions of possible states. The learning rate is so slow and it requires a lot of cache memory to train. We found that it is not practical in our laptops. But it will be the best way to design the model if the computational resources are sufficient enough, because it gives complete information of this game to the AI agent. 

 > Our current setting is to only record directions in state space. For example, one state is whether the food is on the right of the snake head. There are only two possible answers: true or false. In this way, we only have 13 state variables. All variables are binary, so total number of possible states are just a few hundreds. As shown in the video, the training process of this model is very fast. 

 > The reward parameters are set to be +10 if the snake eats the food, -100 if the snake dies, +1 if the snake gets closer to the food, -1 if the snake gets farther from the food. The reason we add ±1 rewards is as follows. What happens if we remove these two rewards? Without them, after a long time of training, we found that the snake becomes lazy and conservative eventually. The AI discovers that the chance of eating the food is too small, compared to the chance of being dead. It is simply too risky to chase the food. As a result, the "smart" AI decides to travel in a loop. Though it can't gain anything, it won't lose anything either. So in the long-term perspective, traveling in a loop is indeed a good strategy.

 > Also as shown in the video, we design a state, named "enclose", to indicate whether the snake encloses itself. By adding this state, the snake learns to avoid putting itself in an enclosed situation. For details, please see our video for a better demonstration.  

6.   *How could you improve your results with future work?*
 > We think our result of training 1 day (1000 episodes) has been closed to converge. That means, training more time probably won't improve the performance significantly. This is because, the state size in our model is only 13. Basically, the AI snake has limited information about the real situation in game. Therefore, the Q-values learned by the agent must be somehow biased, even if we use the DDQN algorithm to eliminate overestimation. One good way to improve the results is by introducing more information. For example, we can use a Convolutional Neural Network (CNN) and pixels as state space.


## **Appendix: DDQN codes**

---

In [ ]:
# Define a Double DQN agent
class DDQNAgent:
    def __init__(self):
        self.env = SnakeGame()
        self.state_size = 13
        self.action_size = 4
        self.EPISODES = 50
        self.memory = deque(maxlen=2000)

        self.gamma = 0.95  # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.batch_size = 500
        self.train_start = 500
        self.TARGET_UPDATE_FREQUENCY = 100

        # create main model and target model
        self.model = OurModel(input_shape=(self.state_size,), action_space=self.action_size)
        self.target_model = OurModel(input_shape=(self.state_size,), action_space=self.action_size)
        # Initialize target model
        self.update_target_model()

    # Function to update the target model to be same with model, after some time interval
    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())

    # Function to remember the state, the action, the reward, and the next state
    def remember(self, state, action, reward, next_state, running):
        self.memory.append((state, action, reward, next_state, running))
        if len(self.memory) > self.train_start:
            if self.epsilon > self.epsilon_min:
                self.epsilon *= self.epsilon_decay

    # Function to pick an action given a state, note that the snake is not allowed to go back
    def act(self, state):
        while True:
            # exploration
            if np.random.rand() <= self.epsilon:
                action_back = random.randrange(self.action_size)
                if action_back == 0 and state[0][2] == 1:
                    continue
                elif action_back == 1 and state[0][3] == 1:
                    continue
                elif action_back == 2 and state[0][0] == 1:
                    continue
                elif action_back == 3 and state[0][1] == 1:
                    continue
                else:
                    return action_back
            # exploitation
            else:
                action_back = np.argmax(self.model.predict(state, verbose=0)[0])
                if action_back == 0 and state[0][2] == 1:
                    action_back = random.randrange(self.action_size)
                    while action_back == 0:
                        action_back = random.randrange(self.action_size)
                elif action_back == 1 and state[0][3] == 1:
                    action_back = random.randrange(self.action_size)
                    while action_back == 1:
                        action_back = random.randrange(self.action_size)
                elif action_back == 2 and state[0][0] == 1:
                    action_back = random.randrange(self.action_size)
                    while action_back == 2:
                        action_back = random.randrange(self.action_size)
                elif action_back == 3 and state[0][1] == 1:
                    action_back = random.randrange(self.action_size)
                    while action_back == 3:
                        action_back = random.randrange(self.action_size)

                return action_back


    # Function to replay the memory and train the network
    def replay(self):
        if len(self.memory) < self.train_start:
            return

        # Randomly sample minibatch from the memory
        minibatch = random.sample(self.memory, min(len(self.memory), self.batch_size))
        state = np.zeros((self.batch_size, self.state_size))
        next_state = np.zeros((self.batch_size, self.state_size))
        action, reward, running = [], [], []

        # assign data into state, next_state, action, reward and running from minibatch
        for i in range(self.batch_size):
            state[i] = minibatch[i][0]
            action.append(minibatch[i][1])
            reward.append(minibatch[i][2])
            next_state[i] = minibatch[i][3]
            running.append(minibatch[i][4])

        # Compute the value function of current state using the primary network
        target = self.model.predict(state, verbose=0)

        # For the next state, use the policy network to select the best action and the target network to evaluate this action
        action_values_next_state = self.model.predict(next_state, verbose=0)
        best_action_next_state = np.argmax(action_values_next_state, axis=1)
        target_next_model = self.target_model.predict(next_state, verbose=0)
        Q_values_next_state = target_next_model[range(self.batch_size), best_action_next_state]

        for i in range(self.batch_size):
            if not running[i]:
                target[i][action[i]] = reward[i]
            else:
                target[i][action[i]] = reward[i] + self.gamma * Q_values_next_state[i]

        self.model.fit(state, target, batch_size=self.batch_size, verbose=0)

    # Function to load the model for testing purpose
    def load(self, name):
        self.model = load_model(name)

    # Function to save the model for testing purpose
    def save(self, name):
        self.model.save(name)

    # Function to train the model
    def training(self):
        total_episodes = []
        total_rewards = []
        total_scores = []
        for e in range(self.EPISODES):
            state = self.env.reset()
            state = np.reshape(state, [1, self.state_size])
            running = True
            i = 0
            while running:
                action = self.act(state)
                running, score, reward, next_state = self.env.run(action)

                next_state = np.reshape(next_state, [1, self.state_size])

                self.remember(state, action, reward, next_state, running)
                state = next_state

                i += reward
                if not running:
                    dateTimeObj = datetime.now()
                    timestampStr = dateTimeObj.strftime("%H:%M:%S")
                    print("episode: {}/{}, score: {}, total reward: {}, e: {:.2}, time: {}".format(e + 1, self.EPISODES, score, i, self.epsilon,
                                                                                 timestampStr))

                    total_episodes.append(e + 1)
                    total_rewards.append(i)
                    total_scores.append(score)

                self.replay()

            # Update target network every TARGET_UPDATE_FREQUENCY episodes
            if e % self.TARGET_UPDATE_FREQUENCY == 0:
                self.update_target_model()

        return total_episodes, total_rewards, total_scores

    # test function if you want to test the learned model
    def test(self):
        self.load("SnakeGame-dqn-training.h5")
        for e in range(self.EPISODES):
            state = self.env.reset()
            state = np.reshape(state, [1, self.state_size])
            running = True
            i = 0
            while running:
                action = np.argmax(self.model.predict(state, verbose=0))
                running, score, reward, next_state = self.env.run(action)
                state = np.reshape(next_state, [1, self.state_size])
                i += reward

                if not running:
                    print("episode: {}/{}, reward: {}".format(e + 1, self.EPISODES, i))
                    break